# Analisador de empresas listadas no Guia Fácil

Tem como objetivo extrair informações relacionadas as empresas de uma determinada região e sua classificação no Google Meu Negócio, se houver.

In [40]:
!pip install requests

In [41]:
!pip install psycopg2-binary

In [42]:
!pip install selenium
!pip install chromedriver-binary

In [43]:
DEBUG = True

def LOG(funcao, mensagem, pulaLinhaAntes = False, pulaLinhaDepois = False):
    if DEBUG == True:
        if pulaLinhaAntes:
            print('\n');
        print('[{}] => {}'.format(funcao, mensagem))
        if pulaLinhaDepois:
            print('\n');

In [44]:
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import chromedriver_binary
import json
import psycopg2
import uuid
import urllib.parse
import requests
import re

In [45]:
def buscaAvaliacaoEmpresaNoGoogle(nomeEmpresa):

    LOG('buscaAvaliacaoEmpresaNoGoogle', 'Iniciado')

    site = 'https://www.google.com/search?q={}'.format(urllib.parse.quote(nomeEmpresa))

    LOG('buscaAvaliacaoEmpresaNoGoogle', 'Buscar empresa no google: {}'.format(site))

    avaliacao = ''
    try:
        options = Options()
        options.add_argument("--headless")
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')
        options.add_argument('start-maximized')  
        options.add_argument('disable-infobars')
        options.add_argument("--disable-extensions")
        service = Service('/home/guiebitt/Workstation/Workspaces/vscode-workspace/cdcoletadadosiot/trabalhos/chromedriver')
        service.start()
        driver = webdriver.Remote(service.service_url, options = options)
        driver.get(site)
        element = driver.find_element(By.CSS_SELECTOR, ".Aq14fc")
        avaliacao = element.text
        driver.quit()
    except:
        avaliacao = ''
        LOG('buscaAvaliacaoEmpresaNoGoogle', 'Erro ao buscar avaliação')

    LOG('buscaAvaliacaoEmpresaNoGoogle', 'Finalizado')

    return avaliacao

In [46]:
def inserirTelefonesEmpresa(empresaId, telefones):
    
    LOG('inserirTelefonesEmpresa', 'Iniciado')

    arquivoTelefones = open('empresas-telefones_joinville-sc.csv', 'a+')

    for telefone in telefones:
        
        idTelefone = uuid.uuid4()
        
        LOG('inserirTelefonesEmpresa', 'UUID do registro: {}'.format(str(idTelefone)))
        
        whatsapp = False
        if telefone['whatsapp'] == 'true':
            whatsapp = True
        
        telefone0800 = False
        if telefone['telefone0800'] == 'true':
            telefone0800 = True

        arquivoTelefones.write("%s;%s;%s;%s;%s;%s;%s\n" % (str(idTelefone), str(empresaId), str(telefone['ddd']), str(telefone['telefone']), str(telefone['telefoneCompleto']), str(whatsapp), str(telefone0800)))

    arquivoTelefones.close()

    LOG('inserirTelefonesEmpresa', 'Finalizado')

    return

In [47]:
def inserirEnderecosEmpresa(empresaId, enderecos):

    LOG('inserirEnderecosEmpresa', 'Iniciado')

    arquivoEnderecos = open('empresas-enderecos_joinville-sc.csv', 'a+')

    for endereco in enderecos:
        
        idEndereco = uuid.uuid4()
        
        LOG('inserirEnderecosEmpresa', 'UUID do registro: {}'.format(str(idEndereco)))

        arquivoEnderecos.write("%s;%s;%s\n" % (str(idEndereco), str(empresaId), str(endereco)))

    arquivoEnderecos.close()

    LOG('inserirEnderecosEmpresa', 'Finalizado')

    return

In [48]:
def inserirEmpresa(empresa):

    LOG('inserirEmpresa', 'Iniciado')

    pessoaJuridica = empresa['pessoaJuridica']
    if pessoaJuridica == False:
        LOG('inserirEmpresa', 'Pessoa fisica: {} nao inserida'.format(empresa['nome']))
        LOG('inserirEmpresa', 'Finalizado')
        return

    avaliacao = buscaAvaliacaoEmpresaNoGoogle('{}, {}'.format(empresa['nome'], empresa['cidade']))

    idEmpresa = uuid.uuid4()
        
    LOG('inserirEmpresa', 'UUID do registro: {}'.format(str(idEmpresa)))
    LOG('inserirEmpresa', 'Avaliação: {}'.format(avaliacao))
    
    arquivo = open('empresas_joinville-sc.csv', 'a+')
    arquivo.write("%s;%s;%s;%s;%s;%s;%s;%s;%s;%s;%s\n" % (str(idEmpresa), str(empresa['nome']), str(empresa['atividade']), str(empresa['emailCliente']), str(empresa['cep']), str(empresa['cidade']), str(empresa['estado']), str(empresa['bairro']), str(empresa['uf']), str(pessoaJuridica), str(avaliacao)))
    arquivo.close()

    inserirTelefonesEmpresa(idEmpresa, empresa['telefones'])
    inserirEnderecosEmpresa(idEmpresa, empresa['enderecos'])

    LOG('inserirEmpresa', 'Registro inserido: {}'.format(empresa['nome']))

    LOG('inserirEmpresa', 'Finalizado')

    return

In [49]:
def inserirEmpresas(empresas):

    LOG('inserirEmpresas', 'Iniciado')

    for empresa in empresas:
        inserirEmpresa(empresa)

    LOG('inserirEmpresas', 'Finalizado')

    return

In [50]:
def buscarEmpresasDaPagina(categoria, cidade, estado, pagina):

    LOG('buscarEmpresasDaPagina', 'Iniciado')

    site = 'https://guiafacil.com/buscaAjax.php?busca={}&cidade={}&tipoBusca=segmento&estado={}&pagina={}'
    site = site.format(categoria, cidade, estado, pagina)

    LOG('buscarEmpresasDaPagina', 'Buscar: {}'.format(site))

    conteudo = urlopen(site).read()
    conteudoJson = json.loads(conteudo)
    resultado = conteudoJson['resultados']

    LOG('buscarEmpresasDaPagina', 'Quantidade de empresas da página: {}'.format(len(resultado)))
    LOG('buscarEmpresasDaPagina', 'Finalizado')

    return resultado

In [51]:
def buscarEmpresas(categoria, cidade, estado):

    LOG('buscarEmpresas', 'Iniciado')

    pagina = 1
    while(True):
        resultadoPagina = buscarEmpresasDaPagina(categoria, cidade, estado, pagina)
        if (len(resultadoPagina) == 0):
            break

        inserirEmpresas(resultadoPagina)
        pagina += 1

    LOG('buscarEmpresas', 'Finalizado')

    return

In [52]:
buscarEmpresas('', 'joinville', 'santa-catarina')

3d053eec
[inserirEmpresa] => Avaliação: 4,3
[inserirTelefonesEmpresa] => Iniciado
[inserirTelefonesEmpresa] => UUID do registro: 144f67ca-c15f-4ccc-b885-63cc19fce1d3
[inserirTelefonesEmpresa] => Finalizado
[inserirEnderecosEmpresa] => Iniciado
[inserirEnderecosEmpresa] => UUID do registro: fa7ff2f0-7fe3-4906-a0ea-d5ee1c979031
[inserirEnderecosEmpresa] => Finalizado
[inserirEmpresa] => Registro inserido: RESTAURANTE E PETISQUEIRA BOX 22
[inserirEmpresa] => Finalizado
[inserirEmpresa] => Iniciado
[buscaAvaliacaoEmpresaNoGoogle] => Iniciado
[buscaAvaliacaoEmpresaNoGoogle] => Buscar empresa no google: https://www.google.com/search?q=23%C2%BA%20GERENCIA%20REGIONAL%20DE%20SAUDE%2C%20Joinville
[buscaAvaliacaoEmpresaNoGoogle] => Finalizado
[inserirEmpresa] => UUID do registro: 564b3f94-3f41-4490-b900-0b9a3da170d7
[inserirEmpresa] => Avaliação: 2,0
[inserirTelefonesEmpresa] => Iniciado
[inserirTelefonesEmpresa] => UUID do registro: be5b9d50-5ec6-4ecc-ab19-1a112ebce58a
[inserirTelefonesEmpresa] 

In [ ]:
72